# Codejam Jupyter Notebook Template

This template allows for interactive prototyping for coding competitions. The `input` function is redefined to read in from a specified file. Additional function code snippets are also defined as required. If these are used, care must be taken to ensure these are copied into the final submission file along with any required imports.

In [1]:
original_input = input

First let's store the original input function so we can use it later if we need

## Helper Functions

### Iteration Functions

In [2]:
from itertools import islice, chain, count, groupby, repeat, starmap, takewhile
from itertools import tee, zip_longest, cycle, filterfalse, combinations
from collections import deque
from operator import mul, itemgetter
import random


def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))


def take_less_than(n, iterable):
    return list(takewhile(lambda x: x < n, iterable))


def prepend(value, iterator):
    "Prepend a single value in front of an iterator"
    # prepend(1, [2, 3, 4]) -> 1 2 3 4
    return chain([value], iterator)


def tabulate(function, start=0):
    "Return function(0), function(1), ..."
    return map(function, count(start))


def tail(n, iterable):
    "Return an iterator over the last n items"
    # tail(3, 'ABCDEFG') --> E F G
    return iter(deque(iterable, maxlen=n))


def consume(iterator, n=None):
    "Advance the iterator n-steps ahead. If n is None, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)


def nth(iterable, n, default=None):
    "Returns the nth item or a default value"
    return next(islice(iterable, n, None), default)


def all_equal(iterable):
    "Returns True if all the elements are equal to each other"
    g = groupby(iterable)
    return next(g, True) and not next(g, False)


def quantify(iterable, pred=bool):
    "Count how many times the predicate is true"
    return sum(map(pred, iterable))


def padnone(iterable):
    """Returns the sequence elements and then returns None indefinitely.

    Useful for emulating the behavior of the built-in map() function.
    """
    return chain(iterable, repeat(None))


def ncycles(iterable, n):
    "Returns the sequence elements n times"
    return chain.from_iterable(repeat(tuple(iterable), n))


def dotproduct(vec1, vec2):
    return sum(map(mul, vec1, vec2))


def flatten(listOfLists):
    "Flatten one level of nesting"
    return chain.from_iterable(listOfLists)


def repeatfunc(func, times=None, *args):
    """Repeat calls to func with specified arguments.

    Example:  repeatfunc(random.random)
    """
    if times is None:
        return starmap(func, repeat(args))
    return starmap(func, repeat(args, times))


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    num_active = len(iterables)
    nexts = cycle(iter(it).__next__ for it in iterables)
    while num_active:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            # Remove the iterator we just exhausted from the cycle.
            num_active -= 1
            nexts = cycle(islice(nexts, num_active))


def partition(pred, iterable):
    'Use a predicate to partition entries into false entries and true entries'
    # partition(is_odd, range(10)) --> 0 2 4 6 8   and  1 3 5 7 9
    t1, t2 = tee(iterable)
    return filterfalse(pred, t1), filter(pred, t2)


def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


def unique_everseen(iterable, key=None):
    "List unique elements, preserving order. Remember all elements ever seen."
    # unique_everseen('AAAABBBCCDAABBB') --> A B C D
    # unique_everseen('ABBCcAD', str.lower) --> A B C D
    seen = set()
    seen_add = seen.add
    if key is None:
        for element in filterfalse(seen.__contains__, iterable):
            seen_add(element)
            yield element
    else:
        for element in iterable:
            k = key(element)
            if k not in seen:
                seen_add(k)
                yield element


def unique_justseen(iterable, key=None):
    "List unique elements, preserving order. Remember only the element just seen."
    # unique_justseen('AAAABBBCCDAABBB') --> A B C D A B
    # unique_justseen('ABBCcAD', str.lower) --> A B C A D
    return map(next, map(itemgetter(1), groupby(iterable, key)))


def iter_except(func, exception, first=None):
    """ Call a function repeatedly until an exception is raised.

    Converts a call-until-exception interface to an iterator interface.
    Like builtins.iter(func, sentinel) but uses an exception instead
    of a sentinel to end the loop.

    Examples:
        iter_except(functools.partial(heappop, h), IndexError)   # priority queue iterator
        iter_except(d.popitem, KeyError)                         # non-blocking dict iterator
        iter_except(d.popleft, IndexError)                       # non-blocking deque iterator
        iter_except(q.get_nowait, Queue.Empty)                   # loop over a producer Queue
        iter_except(s.pop, KeyError)                             # non-blocking set iterator

    """
    try:
        if first is not None:
            yield first()            # For database APIs needing an initial cast to db.first()
        while True:
            yield func()
    except exception:
        pass


def first_true(iterable, default=False, pred=None):
    """Returns the first true value in the iterable.

    If no true value is found, returns *default*

    If *pred* is not None, returns the first item
    for which pred(item) is true.

    """
    # first_true([a,b,c], x) --> a or b or c or x
    # first_true([a,b], x, f) --> a if f(a) else b if f(b) else x
    return next(filter(pred, iterable), default)


def random_product(*args, repeat=1):
    "Random selection from itertools.product(*args, **kwds)"
    pools = [tuple(pool) for pool in args] * repeat
    return tuple(random.choice(pool) for pool in pools)


def random_permutation(iterable, r=None):
    "Random selection from itertools.permutations(iterable, r)"
    pool = tuple(iterable)
    r = len(pool) if r is None else r
    return tuple(random.sample(pool, r))


def random_combination(iterable, r):
    "Random selection from itertools.combinations(iterable, r)"
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(range(n), r))
    return tuple(pool[i] for i in indices)


def random_combination_with_replacement(iterable, r):
    "Random selection from itertools.combinations_with_replacement(iterable, r)"
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.randrange(n) for i in range(r))
    return tuple(pool[i] for i in indices)


def nth_combination(iterable, r, index):
    'Equivalent to list(combinations(iterable, r))[index]'
    pool = tuple(iterable)
    n = len(pool)
    if r < 0 or r > n:
        raise ValueError
    c = 1
    k = min(r, n - r)
    for i in range(1, k + 1):
        c = c * (n - k + i) // i
    if index < 0:
        index += c
    if index < 0 or index >= c:
        raise IndexError
    result = []
    while r:
        c, n, r = c * r // n, n - 1, r - 1
        while index >= c:
            index -= c
            c, n = c * (n - r) // n, n - 1
        result.append(pool[-1 - n])
    return tuple(result)


### Primes

In [3]:
from collections import defaultdict


def primes():
    yield 2

    composites = defaultdict(list)
    i = 1
    while True:
        i += 2
        if i in composites:
            composite = composites.pop(i)
            for j in composite:
                composites[i + 2 * j].append(j)
        else:
            composites[3 * i].append(i)
            yield i

In [4]:
from itertools import product
from functools import reduce
from operator import mul


def factors(n, prime_list=None):
    if n < 1:
        raise ValueError
    factor_dict = defaultdict(int)
    if prime_list is None:
        prime_list = primes()

    for p in prime_list:
        while n % p == 0:
            n /= p
            factor_dict[p] += 1
        if n == 1:
            break
    factor_list = []

    for powers in product(*[list(range(j + 1)) for j in factor_dict.values()]):
        factor_list.append(reduce(mul, [p**powers[i] for i, p in enumerate(factor_dict)], 1))

    return sorted(factor_list)

In [5]:
from collections import defaultdict


def prime_factors(n, prime_list=None):
    if n < 2:
        raise ValueError
    factor_dict = defaultdict(int)
    if prime_list is None:
        prime_list = primes()

    for p in prime_list:
        while n % p == 0:
            n /= p
            factor_dict[p] += 1
        if n == 1:
            break

    return factor_dict

In [6]:
def is_prime(n, prime_list=None, prime_set=None):
    if n < 2:
        return False
    if prime_set is not None and n in prime_set:
        return True
    if prime_list is None:
        prime_list = primes()
    for p in prime_list:
        if p * p > n:
            break
        if n % p == 0:
            return False
    return True

In [7]:
def gcd(a, b):
    # implements Euclid's algorithm
    if b > a:
        a, b = b, a
    q = int(a // b)
    r = a - q * b
    while r != 0:
        a, b = b, r
        q = int(a // b)
        r = a - q * b
    return int(b)

In [8]:
def is_coprime(j, k):
    return gcd(j, k) == 1

In [9]:
from functools import reduce


def chinese_remainder(D, Y):
    # Returns the value x (mod M) such that when x is divided by any di in
    # the list of coprime integers D the remainder is the corresponding yi in
    # the list of remainders Y. M is the product of the Ds.

    M = reduce(lambda x, y: x * y, D, 1)
    B = [int(M / d) for d in D]
    A = []
    for d, b in zip(D, B):
        for a in range(1, d):
            if a * b % d == 1:
                A.append(a)
                break

    x = sum([a * b * y for a, b, y in zip(A, B, Y)])
    return x % M

## Input redirection

Run this if you want to use the original input

In [10]:
input = original_input

Otherwise set input to redirect to an input file

In [11]:
def input_from_file(filename):
    for line in open(filename):
        yield line.strip()

Run this again to reset the file

In [81]:
input_file = "edgy_baking.sample.in"

In [82]:
file_input = input_from_file(input_file)
input = lambda: next(file_input)

## Start coding here

Place any import statements you need here

In [83]:
from math import sqrt

Define any helper functions you need here

Define the main function you're going to use here

In [84]:
def run(cookies, p):
    # This will probably require some input arguments
    W = [cookie[0] for cookie in cookies]
    H = [cookie[1] for cookie in cookies]
    original_perimeter = sum([2*(w + h) for w, h in zip(W, H)])
    target_additional = p - original_perimeter
    upper_bound = sum([2*sqrt(w**2 + h**2) for w, h in zip(W, H)])
    lower_bound = 0
    intervals = []
    previous_interval = [0, 0]
    for w, h in zip(W, H):
        new_interval = [previous_interval[0] + 2*w, previous_interval[1] + 2*sqrt(w**2 + h**2)]
        if target_additional < new_interval[0]:
            upper_bound = new_interval[0]
            break
        elif target_additional <= new_interval[1]:
            upper_bound = target_additional
            lower_bound = target_additional
            break
        else:
            lower_bound = new_interval[1]
            
        intervals.append(new_interval)
        previous_interval = new_interval
    if target_additional - lower_bound < upper_bound - target_additional:
        return original_perimeter + lower_bound
    else:
        return original_perimeter + upper_bound

Use this as the main body function to run

In [85]:
T = int(input())
for t in range(1, T + 1):
    # There will probably be some additional inputs
    N, P = [int(i) for i in input().split()]
    cookies = [[int(i) for i in input().split()] for n in range(N)]
    cookies = [[w, h] if w < h else [h, w] for w, h in cookies]
    result = run(cookies, P)
    print("Case #{}: {}".format(t, result))

Case #1: 6.82842712474619
Case #2: 920
Case #3: 32
Case #4: 240


## Prep solution file

- Grab any helper functions and import statements that you need
- Stick the main body function inside an `if __name__ == "__main__":` block
- This will allow importing the individual functions from the solution file

## Sandbox

In [14]:
N = 2
P = 920

In [15]:
W = 50
H = 120

In [16]:
if W > H:
    W, H = H, W

In [26]:
original_perimeter = (W + H)*2*2

In [27]:
original_perimeter

680

In [28]:
target_additional = P - original_perimeter

In [29]:
target_additional

240

In [30]:
from math import sqrt

In [31]:
min_edge = 2 * W
max_edge = 2 * sqrt(W**2 + H**2)

In [32]:
min_edge, max_edge

(100, 260.0)

In [33]:
intervals = [[min_edge, max_edge], [2*min_edge, 2*max_edge]]

In [34]:
intervals

[[100, 260.0], [200, 520.0]]

In [ ]:
for interval in intervals